## Write your own GNN module

> `Message Passing`는 node의 representation을 계산하기 위해 이웃 node에서 전달받는 과정을 의미합니다. 이때 주변 node에서 전달하는 정보를 `Message`라고 한다. 가장 대표적인 모델로는 `GraphSAGE`가 있습니다.

In [2]:
import dgl 
import torch 
import torch.nn as nn 
import torch.nn.functional as F

### $$ m^{(l)}_{u \rightarrow v} = M^{(l)} \left ( h^{(l-1)}_v h^{(l-1)}_u, e^{(l-1)}_{u \rightarrow v} \right ) $$
### $$ m^{(l)}_v = \sum_{u \in \mathcal{N}(v)} m^{(l)}_{u \rightarrow v} $$ 
### $$ h^{(l)}_v = U^{(l)} \left ( h^{(l-1)}_v , m^{(l)}_v \right)  $$ 
### $$ h^k_{\mathcal{N}(v)} \leftarrow \text{Average} \{ h^{k-1}_u, \forall u \in \mathcal{N}(v) \}$$ 
### $$ h^k_v \leftarrow \text{ReLU}( W^k \cdot \text{CONCAT} (h^{k-1}_v, h^k_{\mathcal{N}(v)})) $$
$ m^{(l)}_{u \rightarrow v}$는 node $u$에서 node $v$로 전달하는 `Message`를 의미합니다. $m^{(l)}_v$는 node $v$의 `Message`를 의미하며, 이웃 node들로 전달받은 `Message`의 합으로 계산됩니다. 전달받은 `Message`를 Aggregation(mean)하고 비선형 활성화 함수를 사용하여 최종 $h^k_v$를 출력합니다. 

In [4]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [5]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [6]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.951, val acc: 0.072 (best 0.072), test acc: 0.091 (best 0.091)


c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 5, loss: 1.890, val acc: 0.286 (best 0.286), test acc: 0.313 (best 0.313)
In epoch 10, loss: 1.769, val acc: 0.624 (best 0.624), test acc: 0.658 (best 0.658)
In epoch 15, loss: 1.578, val acc: 0.596 (best 0.624), test acc: 0.620 (best 0.658)
In epoch 20, loss: 1.320, val acc: 0.604 (best 0.624), test acc: 0.591 (best 0.658)
In epoch 25, loss: 1.018, val acc: 0.650 (best 0.650), test acc: 0.628 (best 0.628)
In epoch 30, loss: 0.715, val acc: 0.706 (best 0.706), test acc: 0.685 (best 0.685)
In epoch 35, loss: 0.461, val acc: 0.730 (best 0.730), test acc: 0.721 (best 0.721)
In epoch 40, loss: 0.280, val acc: 0.742 (best 0.742), test acc: 0.726 (best 0.726)
In epoch 45, loss: 0.166, val acc: 0.742 (best 0.742), test acc: 0.734 (best 0.726)
In epoch 50, loss: 0.100, val acc: 0.748 (best 0.748), test acc: 0.734 (best 0.734)
In epoch 55, loss: 0.063, val acc: 0.752 (best 0.754), test acc: 0.735 (best 0.734)
In epoch 60, loss: 0.043, val acc: 0.752 (best 0.754), test acc: 0.739 (best 

## More customization

In [7]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [ ]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

## Even more customization by user-defined function

In [ ]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

In [ ]:
def mean_udf(nodes):
    return {'h_N': nodes.mailbox['m'].mean(1)}